# Libraries

In [ ]:
import os
import warnings

import matplotlib.pyplot as pyplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import (
    accuracy_score,
    classification_report,
    confusion_matrix,
    precision_score,
    r2_score,
    recall_score,
)
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.svm import LinearSVC
from sklearn.tree import DecisionTreeClassifier

warnings.filterwarnings('ignore')

In [ ]:
path = '../Artifacts/postoperativedata.csv'
df = pd.read_csv(path)
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
for i in df.columns:
    print(df[i].unique())

In [ ]:
df["COMFORT"] = df["COMFORT"].replace("?", "10")

In [ ]:
df["COMFORT"] = pd.to_numeric(df["COMFORT"])

In [ ]:
df.isnull().sum()

In [ ]:
le = LabelEncoder()
deviceEnc = pd.DataFrame(le.fit_transform(df['L-SURF']), columns=['L-SURF'])

dataEnc = pd.concat([df, deviceEnc], axis=1)
dataEnc.head()

In [ ]:
lightEnc = pd.DataFrame(le.fit_transform(dataEnc['L-CORE']), columns=['L-CORE'])
lightEnc.index = dataEnc.index

dataEnc2 = pd.concat([dataEnc, lightEnc], axis=1)
dataEnc2.head()

In [ ]:
sns.countplot(data=df, x="decision ADM-DECS")

# Splitting the data 

In [ ]:
X = df[:, :-1]
y = df[:, -1]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Model Building

In [ ]:
models = {}
models["Logistic Regression"] = LogisticRegression()
models["Stochastic Gradient Descent"] = SGDClassifier(
    loss="modified_huber", shuffle=True, random_state=101
)
models["Support Vector Machines"] = LinearSVC()
models["Decision Trees"] = DecisionTreeClassifier()
models["Random Forest"] = RandomForestClassifier()
models["Naive Bayes"] = GaussianNB()
models["K-Nearest Neighbor"] = KNeighborsClassifier()

In [ ]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    print("Model:\t{}".format(model))

 # Training the models and calculating the acuuaracy

In [ ]:
accuracy, precision, recall, r2 = {}, {}, {}, {}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train, y_train)

    predictions = model.predict(X_test)

    accuracy[key] = accuracy_score(predictions, y_test)
    precision[key] = precision_score(predictions, y_test)
    recall[key] = recall_score(predictions, y_test)
    r2[key] = r2_score(predictions, X_test)

# Accuaracy of models build 

In [ ]:
df_model = pd.DataFrame(
    index=models.keys(), columns=["Accuracy", "Precision", "Recall"]
)
df_model["Accuracy"] = accuracy.values()
df_model["Precision"] = precision.values()
df_model["Recall"] = recall.values()
df_model["R2 Score"] = r2_score.values()

df_model

In [ ]:
ax = df_model.plot.barh()
ax.legend(
    ncol=len(models.keys()), bbox_to_anchor=(0, 1), loc="lower left", prop={"size": 14}
)
plt.tight_layout()